# MGMTMSA 403 Homework 2: Portfolio Optimization


**Group Members: Guangying Pan, Yijie Fu**

## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


## Question 1

**(a) For Model 1, write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.**

### Model 1

**optimal risk**: 1.77493264e-04

**solver time**: 0.02 seconds

**Weight**:

315 ("MSFT"): 0.23711755540145132

216 ("GS"): 0.025859609077563107

372 ("PG"): 1.643079012001778e-10

388 ("SCHP"): 0.7370228353566646

In [2]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
w = mod.addVars(390)

# Construct constraints
# Weights add up to 1
mod.addConstr(sum(w[i] for i in tickind) == 1)

# Weights are greater than 0
for i in tickind:
    mod.addConstr(w[i] >= 0)

# Expected return should be at least r
mod.addConstr(sum(w[i]*avg_return[i] for i in tickind) >= 0.005)

# Create the objective function, and set it to be minimized
mod.setObjective(sum(w[i]*w[j]*C[i,j] for i in tickind for j in tickind), GRB.MINIMIZE)

mod.update()

mod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 390 columns and 12 nonzeros
Model fingerprint: 0x4342ba91
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 386 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal    

In [3]:
# Extract the solution status. 
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")

    # print optimized value obtained at the optimal solution
    print(f"\nOptimized Value: {mod.objval}")

    # Extract the optimal values of the decision variables
    for i in tickind:
        print(f"\n {i}: Weight: {w[i].X}")

Solved to optimality

Optimized Value: 0.00017749326422824104

 315: Weight: 0.23711755540145132

 216: Weight: 0.025859609077563107

 372: Weight: 1.643079012001778e-10

 388: Weight: 0.7370228353566646


**(b) For Model 2, write down the optimal risk and solver time.**

### Model 2

**optimal risk**: 2.87917525e-05
    
**solver time**: 0.04 seconds

In [4]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
w = mod.addVars(390)

# Construct constraints
# Weights add up to 1
mod.addConstr(sum(w[i] for i in range(390)) == 1)

# Weights are greater than 0
for i in range(390):
    mod.addConstr(w[i] >= 0)

# Expected return should be at least r
mod.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)

# Create the objective function, and set it to be minimized
mod.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xdb4291c8
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 -

**(c) For Model 3, report the optimal risk, solver time, and the ticker and weight on each of the
four stocks selected by the model.**

### Model 3

**optimal risk**: 6.753470760728e-05

**solver time**: 34.94 seconds

**Weights**:

CME: 0.12641141929490474

LLY: 0.07547619035437456

NVDA: 0.043753706728431804

BND: 0.754358683622289

In [5]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
w = mod.addVars(390)

# Add decision variable: whether to choose the stock or not
X = mod.addVars(390, vtype=GRB.BINARY)


# Construct constraints
# Weights add up to 1
mod.addConstr(sum(w[i] for i in range(390)) == 1)

# Weights are greater than 0
for i in range(390):
    mod.addConstr(w[i] >= 0)

# Expected return should be at least r
mod.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)

# selects at most 4
mod.addConstr(sum(X[i] for i in range(390)) <= 4)

# when X is 0, then w is 0; when X is 1, then w is positive
for i in range(390):
    mod.addConstr(X[i] >= w[i])

# Create the objective function, and set it to be minimized
mod.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xf480024a
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.04s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [6]:
# Extract the solution status. 
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")

    # print optimized value obtained at the optimal solution
    print(f"\nOptimized Value: {mod.objval}")

    # Extract the optimal values of the decision variables
    for i in range(390):
        if X[i].X == 1:
            ticker_name = tickers[i]
            print(f"\n {ticker_name}: Weight: {w[i].X}")

Solved to optimality

Optimized Value: 6.753470760728118e-05

 CME: Weight: 0.12641141929490474

 LLY: Weight: 0.07547619035437456

 NVDA: Weight: 0.043753706728431804

 BND: Weight: 0.754358683622289


## Question 2

**(a) Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.**

Optimal risk in Model 1 is higher than Model 2 due to a lack of diversification. With only 4 stocks, portfolio is more exposed to the specific risks associated with each of these stocks (like sector-specific risks, company-specific risks, etc.), whereas a portfolio with 390 stocks is more likely to be spread across different sectors and types of companies, thereby diluting these specific risks.

**(b) Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences.**

Optimal risk in Model 2 is lower than Model 3. In Model 2, weights are distributed across all 390 stocks - the portfolio is diversified and thus when attempting to minimize risk, we get a lower optimal risk. In Comparison, Model 3 is more selective and less diversified. 

## Question 3

**(a) Set Gurobi to terminate after 3 seconds by including XYZ.Params.TimeLimit = 3.0 in your code for Model 3, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?**

The objective function value at termination compare the optimal value obtained in question 1c) are the same.

In [7]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
w = mod.addVars(390)

# Add decision variable: whether to choose the stock or not
X = mod.addVars(390, vtype=GRB.BINARY)


# Construct constraints
# Weights add up to 1
mod.addConstr(sum(w[i] for i in range(390)) == 1)

# Weights are greater than 0
for i in range(390):
    mod.addConstr(w[i] >= 0)

# Expected return should be at least r
mod.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)

# selects at most 4
mod.addConstr(sum(X[i] for i in range(390)) <= 4)

# when X is 0, then w is 0; when X is 1, then w is positive
for i in range(390):
    mod.addConstr(X[i] >= w[i])

# Create the objective function, and set it to be minimized
mod.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

mod.update()

mod.Params.TimeLimit = 3.0

mod.optimize()

Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xf480024a
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Obje

**(b) Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1 in your code for Model 3, where ’XYZ’ is the name of your model. (Note: The default gap in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in question 1c)?**

The solver time in 3(b) is shorter as compared with the solution time obtained in question 1c).

In [8]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
w = mod.addVars(390)

# Add decision variable: whether to choose the stock or not
X = mod.addVars(390, vtype=GRB.BINARY)


# Construct constraints
# Weights add up to 1
mod.addConstr(sum(w[i] for i in range(390)) == 1)

# Weights are greater than 0
for i in range(390):
    mod.addConstr(w[i] >= 0)

# Expected return should be at least r
mod.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)

# selects at most 4
mod.addConstr(sum(X[i] for i in range(390)) <= 4)

# when X is 0, then w is 0; when X is 1, then w is positive
for i in range(390):
    mod.addConstr(X[i] >= w[i])

# Create the objective function, and set it to be minimized
mod.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

mod.update()

mod.Params.MIPGap = 0.1

mod.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xf480024a
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objec